In [13]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# 임시 전처리: ToTensor()만 사용하여 데이터를 로드 (정규화 없이)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(),  # 흑백 이미지를 만들기 위한 변환 추가
    transforms.ToTensor()
])

# 데이터셋 로드 (train 폴더 내의 각 클래스 폴더를 자동으로 라벨링)
train_dataset = datasets.ImageFolder(root="C:\\Users\\BIG3-04\\Desktop\\한글인식2", transform=transform)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# 데이터셋의 평균과 표준 편차 계산
def calculate_mean_std(loader):
    mean = torch.zeros(1)
    std = torch.zeros(1)
    total_samples = 0

    for images, _ in loader:
        # 배치 크기와 채널 수
        batch_samples = images.size(0)
        
        # 흑백 이미지는 단일 채널이므로 0번째 채널만 사용
        mean[0] += images[:, 0, :, :].mean().item() * batch_samples
        std[0] += images[:, 0, :, :].std().item() * batch_samples

        total_samples += batch_samples

    mean /= total_samples
    std /= total_samples

    return mean, std

mean, std = calculate_mean_std(train_loader)
print(f'Mean: {mean}')
print(f'Std: {std}')



Mean: tensor([0.6596, 0.6411, 0.6329])
Std: tensor([0.1305, 0.1439, 0.1389])


In [ ]:
import os
import torch
import json
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
from torch import nn, optim
from PIL import Image

# 데이터 변환
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # 흑백 이미지로 변환
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.6401], std=[0.1575]),
])

augment_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # 흑백 이미지로 변환
    transforms.RandomRotation(30),  # 랜덤 회전
    transforms.Resize((224, 224)),  # 크기 조정
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.6459], std=[0.1376]),
])

# 데이터셋 로드
dataset = datasets.ImageFolder(root="C:\\Users\\BIG3-04\\Desktop\\한글인식", transform=transform)
class_names = dataset.classes

# 클래스 이름을 저장할 경로
class_names_save_path = os.path.join("C:\\Users\\BIG3-04\\Desktop\\모델", 'class_names.json')
with open(class_names_save_path, 'w', encoding='utf-8') as f:
    json.dump(class_names, f, ensure_ascii=False, indent=4)

# 데이터셋 크기 확인
dataset_size = len(dataset)
train_size = int(0.8 * dataset_size)
test_size = dataset_size - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
train_dataset.dataset.transform = augment_transform

# 데이터 로더
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 모델 로드 및 수정
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names))

# 손실 함수 및 옵티마이저
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9 ,  weight_decay=1e-4)

# 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Early Stopping 설정
early_stopping_patience = 5
best_val_loss = float('inf')
patience_counter = 0

# 학습 및 검증 손실과 정확도 기록
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

def calculate_accuracy(loader, model):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

# 모델 학습
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    train_loss = running_loss / len(train_loader)
    train_losses.append(train_loss)
    
    train_accuracy = calculate_accuracy(train_loader, model)
    train_accuracies.append(train_accuracy)
    
    val_loss = 0.0
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        val_loss += loss.item()
    
    val_loss /= len(test_loader)
    val_losses.append(val_loss)
    
    val_accuracy = calculate_accuracy(test_loader, model)
    val_accuracies.append(val_accuracy)
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Train Acc: {train_accuracy:.4f}, Val Acc: {val_accuracy:.4f}')
    
    # Early Stopping 체크
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        # 모델 저장
        torch.save(model.state_dict(), "C:\\Users\\BIG3-04\\Desktop\\모델\\finalLLL_model.pth")
    else:
        patience_counter += 1
    
    if patience_counter >= early_stopping_patience:
        print("Early stopping")
        break